In [ ]:
from safetensors.torch import load_file

model_ckpt = load_file("../checkpoints/svd.safetensors")

print(model_ckpt.keys())

In [1]:
from safetensors.torch import load_file
import torch

torch.load(
    "/fsx/behavioural_computing_data/face_generation_data/1000actors_nsv/audio_emb/A435_C003_102997_001_wavlm_emb.pt"
).shape

torch.Size([432, 2, 768])

In [2]:
from torchvision.io import read_video

video, audio, info = read_video(
    "/fsx/behavioural_computing_data/face_generation_data/AA_processed/part_8/video_crop/D550_C047_0106XS_001_output_output.mp4"
)
video.shape, info, video.shape[0] / 60

/data/home/antoni/miniconda3/envs/svd/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(torch.Size([0, 1, 1, 3]), {}, 0.0)

In [1]:
import torchaudio

audio, sr = torchaudio.load(
    "/fsx/behavioural_computing_data/face_generation_data/AA_processed/part_8/audio/D550_C047_0106XS_001.wav"
)
audio.shape, sr, audio.shape[-1] / sr

(torch.Size([1, 1912491]), 16000, 119.5306875)

In [ ]:
import torch

print(
    torch.load(
        "/fsx/behavioural_computing_data/face_generation_data/CelebV-Text/audio_emb/1/-6yEyJRIUGc_5_0_0_170_beats_emb.pt"
    ).shape
)

from safetensors.torch import load_file

torch.load(
    "/fsx/behavioural_computing_data/face_generation_data/CelebV-Text/audio_emb/1/-6yEyJRIUGc_5_0_0_170_beats_emb.pt"
).shape

In [ ]:
load_file(
    "/fsx/behavioural_computing_data/face_generation_data/CelebV-Text/audio_emb/1/-6yEyJRIUGc_5_0_0_170_wav2vec2_emb.safetensors"
)["audio"].shape

In [ ]:
import math


def max_divisions(a, b):
    if b == 1:
        return float("inf")  # Dividing by 1 will never reduce the number
    return math.floor(math.log(a, b))


# Example usage:
a = 100
b = 14
print(max_divisions(a, b))  # Output: 6

In [ ]:
import torch

ckpt = torch.load(
    "../logs/2024-08-21T12-15-52_example_training-svd_interpolation_no_emb/checkpoints/last.ckpt/checkpoint/mp_rank_00_model_states.pt",
    map_location="cpu",
)

In [ ]:
for k in ckpt["module"].keys():
    if "learned_mask" in k:
        print(k)
        print(ckpt["module"][k])
        print(model_ckpt[k])

In [ ]:
import torch

state_dict = torch.load(
    "../logs/2024-05-30T18-28-54_example_training-svd_image/checkpoints/last.ckpt/checkpoint/mp_rank_00_model_states.pt",
    map_location="cpu",
)
state_dict.keys()

In [ ]:
state_dict["module"].keys()
# Take only weights that contain the word "diffusion_model" and keep only what comes after the word "diffusion_model" but including the word "diffusion_model"
new_dict = {}
for k, v in state_dict["module"].items():
    if "diffusion_model" in k:
        new_dict["diffusion_model" + k.split("diffusion_model")[1]] = v
new_dict.keys()

In [ ]:
import torch

ckpth_path = "/vol/bitbucket/abigata/checkpoints/trainstep_checkpoints/last.ckpt/checkpoint/mp_rank_00_model_states.pt"
ckpt = torch.load(ckpth_path)

print(ckpt["state_dict"].keys())

In [ ]:
# create new weights for modified model
new_weights = {}
for k, v in model_ckpt.items():
    # Remove model weights
    # if k.startswith("model."):
    #     continue
    # Since embedder 1 and 2 have been removed, we need to remove the corresponding weights and change number of following embedders
    if "embedders.1" in k or "embedders.2" in k:
        print("chybrax")
        continue
    if "embedders.3" in k:
        new_weights[k.replace("embedders.3", "embedders.1")] = v
        continue
    new_weights[k] = v

In [ ]:
# save new weights
from safetensors.torch import save_file

save_file(new_weights, "../checkpoints/svd_no_emb.safetensors")

In [ ]:
import torch.nn.functional as F


# Function to ensure all tensors are [1, 640]
def ensure_shape(tensors, target_length=640):
    processed_tensors = []
    for tensor in tensors:
        current_length = tensor.shape[1]
        diff = current_length - target_length
        assert abs(diff) <= 5, f"Expected shape {target_length}, but got {current_length}"
        if diff < 0:
            # Calculate how much padding is needed
            padding_needed = target_length - current_length
            # Pad the tensor
            padded_tensor = F.pad(tensor, (0, padding_needed))
            processed_tensors.append(padded_tensor)
        elif diff > 0:
            # Trim the tensor
            trimmed_tensor = tensor[:, :target_length]
            processed_tensors.append(trimmed_tensor)
        else:
            # If it's already the correct size
            processed_tensors.append(tensor)
    return torch.cat(processed_tensors)

In [ ]:
from decord import AVReader
from decord import cpu, gpu
import decord

decord.bridge.set_bridge("torch")

import torch

av = AVReader(
    "/vol/paramonos2/projects/antoni/code/Personal/generative-models/outputs/simple_video_sample/svd/000096_gt.mp4",
    ctx=cpu(0),
    sample_rate=16000,
)
# To access both the video frames and corresponding audio samples
audio, video = av[0:8]
# for a in audio:
#     print(a.shape)
# audio = torch.cat(audio, dim=0)
# # Each element in audio will be a batch of samples corresponding to a frame of video
# print('Frame #: ', len(audio), len(video))
# print('Shape of the audio samples of the first frame: ', audio.shape, len(audio))
# print('Shape of the first frame: ', video[0].shape)
# Similarly, to get a batch
audio2, video2 = av.get_batch(range(0, len(av)))
audio2 = ensure_shape(audio2)
print("Shape of the audio samples of the first frame: ", audio2.shape)

In [ ]:
av._AVReader__video_reader

In [ ]:
from einops import rearrange

raw_audio = rearrange(audio2, "t c -> (t c)")
raw_audio.shape

In [ ]:
from scipy.io.wavfile import write

write("test.wav", 16000, raw_audio.numpy())

In [ ]:
# Plot video
import moviepy.editor as mp

mp.VideoFileClip(
    "/vol/paramonos2/projects/antoni/code/Personal/generative-models/outputs/simple_video_sample/svd/000059.mp4"
).ipython_display()

In [ ]:
import webdataset as wds
import glob

tar_files = glob.glob("/data/122-2/Datasets/CREMA/webdataset/train/batch_*.tar")


def preprocess_video(video):
    # Your video preprocessing code here
    return video


def preprocess_audio(audio):
    # Your audio preprocessing code here
    return audio


# Define how each type of file should be processed
def process_sample(sample):
    # Assuming the keys are 'video.mp4' and 'audio.wav' in your tar files
    video, audio = sample["mp4"], sample["wav"]

    # Apply your preprocessing functions
    processed_video = preprocess_video(video)
    processed_audio = preprocess_audio(audio)

    return processed_video, processed_audio


def custom_decoder(sample):
    # Add custom decoding or handling for specific file types
    # For example, if sample contains a '.wav' key, decode it appropriately
    print(sample.keys())
    return sample


ds = wds.WebDataset(tar_files).map(custom_decoder)
for sample in ds:
    print(sample.keys())
    break

In [ ]:
import os
import ffmpeg
import tempfile


class ResolutionSubsampler:
    """
    Adjusts the resolution of the videos to the specified height and width.

    Args:
        video_size (int): Target resolution of the videos.
        resize_mode (list[str]): List of resize modes to apply. Possible options are:
            scale: scale video keeping aspect ratios (currently always picks video height)
            crop: center crop to video_size x video_size
            pad: center pad to video_size x video_size
    """

    def __init__(self, video_size, resize_mode):
        self.video_size = video_size
        self.resize_mode = resize_mode

    def __call__(self, streams):
        video_bytes = streams["video"]
        subsampled_bytes = []
        with tempfile.TemporaryDirectory() as tmpdir:
            with open(os.path.join(tmpdir, "input.mp4"), "wb") as f:
                f.write(video_bytes)
            try:
                _ = ffmpeg.input(f"{tmpdir}/input.mp4")
                if "scale" in self.resize_mode:
                    _ = _.filter("scale", -2, self.video_size)
                if "crop" in self.resize_mode:
                    _ = _.filter("crop", w=self.video_size, h=self.video_size)
                if "pad" in self.resize_mode:
                    _ = _.filter("pad", w=self.video_size, h=self.video_size)
                _ = _.output(f"{tmpdir}/output.mp4", reset_timestamps=1).run(capture_stdout=True, quiet=True)
            except Exception as err:  # pylint: disable=broad-except
                return [], None, str(err)

            with open(f"{tmpdir}/output.mp4", "rb") as f:
                subsampled_bytes = f.read()
        streams["video"] = subsampled_bytes
        return streams

In [ ]:
def custom_decoder(sample):
    # Add custom decoding or handling for specific file types
    # For example, if sample contains a '.wav' key, decode it appropriately
    for key in sample.keys():
        if key.endswith("video"):
            sample[key] = wds.torch_video("mp4", sample[key])
        if key.endswith("audio"):
            sample[key] = wds.torch_audio("wav", sample[key])
        elif key.endswith("audio_emb"):
            sample[key] = wds.torch_loads(sample[key])
    return sample


import webdataset as wds

i = 0
ds = (
    wds.WebDataset("/data/122-2/Datasets/CREMA/webdataset/train/out-{000000..000006}.tar")
    .rename(video="mp4;video", audio="wav;audio", audio_emb="pt;audio_emb")
    .map(ResolutionSubsampler(256, ["scale"]))
    .map(custom_decoder)
)
for sample in ds:
    print(sample.keys())
    i += 1
    # if i > 10:
    break
print(i)

In [ ]:
import webdataset as wds

i = 0
ds = wds.WebDataset("/data/122-2/Datasets/CREMA/webdataset/train/out-{000000..000006}.tar").decode("torchl")
for sample in ds:
    print(sample.keys())
    # print(sample['mp4'][0].shape)
    print(sample["pt"].shape)
    i += 1
    # if i > 10:
    break
print(i)

In [ ]:
with open("/data/122-2/Datasets/CREMA/s32/audio/1032_IEO_FEA_LO_emb.pt", "rb") as f:
    video = f.read()

In [ ]:
import webdataset as wds

wds.autodecode.decoders
wds.autodecode.imagespecs

In [ ]:
from sdata import create_dataset, create_loader

from omegaconf import OmegaConf
from tqdm import tqdm

config = OmegaConf.load("/vol/paramonos2/projects/antoni/code/Personal/generative-models/notebooks/data.yaml")

# build config
datapipeline = create_dataset(**config.dataset)

i = 0
for sample in tqdm(datapipeline, desc="Loading dataset"):
    i += 1
    break

print(i)

In [ ]:
datapipeline.__dict__

In [ ]:
# Plot cond_frames
import matplotlib.pyplot as plt
import numpy as np

cond = ((sample["cond_frames"].permute(1, 2, 0).numpy() + 1) / 2).clip(0, 1) * 255
cond = cond.astype(np.uint8)
plt.imshow(cond)
print(sample["cond_aug"])

In [ ]:
import torch
from torch import einsum

scale = 1
num_audio_frames = 28
q = torch.randn(1, 14, 320)
k = torch.randn(1, num_audio_frames, 320)
v = torch.randn(1, num_audio_frames, 320)

sim = einsum("b i d, b j d -> b i j", q, k) * scale
# del q, k

# attention, what we cannot get enough of
sim = sim.softmax(dim=-1)

out = einsum("b i j, b j d -> b i d", sim, v)
out.shape

In [ ]:
import torch

checkpoint_path = "/vol/bitbucket/abigata/mp_rank_00_model_states.pt"
checkpoint = torch.load(checkpoint_path)

print(checkpoint.keys())

In [ ]:
try:
    # Attempt to load the .pt file
    data = torch.load(checkpoint_path)
    print("File loaded successfully. Integrity check passed.")
except Exception as e:
    # Handle exceptions that indicate file loading issues
    print(f"Failed to load file: {e}")

In [ ]:
checkpoint = data

In [ ]:
from safetensors.torch import save_file

sd = {k.replace("_forward_module.", ""): v for k, v in checkpoint["module"].items()}
save_file(sd, checkpoint_path.replace(".pt", ".safetensors"))

In [ ]:
for k in checkpoint["module"]:
    if "model.diffusion_model.input_blocks.1.0.time_stack.emb_layers.1.weight" in k:
        print(k)

In [ ]:
import random


def select_indices(max_frames, n):
    # Step 1: Randomly select the first index
    index1 = random.randint(0, max_frames)

    # Step 2: Determine the valid range for the second index
    valid_range = list(range(0, index1 - n)) + list(range(index1 + n + 1, max_frames + 1))

    # Check if the valid range is not empty
    if not valid_range:
        raise ValueError("The range is too small for these constraints!")

    # Step 3: Randomly select the second index from the valid range
    index2 = random.choice(valid_range)

    return index1, index2


# Usage example
max_frames = 100  # for example, 100 frames
n = 10  # must be at least 10 indices apart
indices = select_indices(max_frames, n)
print(indices)

In [ ]:
from safetensors import safe_open

path = "/fsx/rs2517/data/HDTF/cropped_videos_original/WRA_ToddYoung_000_video_512_latent.safetensors"
tensors = {}
with safe_open(path, framework="pt") as f:
    tensor_slice = f.get_slice("latents")
    print(tensor_slice.get_shape())
    tensor = tensor_slice[0:]

In [ ]:
for i in range(0, 25):
    for j in range(i, 25):
        if tensor[6250 + i].isclose(tensor[6250 + j]).all():
            print(i)

In [ ]:
def get_audio_indexes(main_index, n_audio_frames, max_len):
    # Get indexes for audio from both sides of the main index
    audio_ids = []
    # get audio embs from both sides of the GT frame
    audio_ids += [0] * max(n_audio_frames - main_index, 0)
    for i in range(max(main_index - n_audio_frames, 0), min(main_index + n_audio_frames + 1, max_len)):
        # for i in range(frame_ids[0], min(frame_ids[0] + self.n_audio_motion_embs + 1, n_frames)):
        audio_ids += [i]
    audio_ids += [max_len - 1] * max(main_index + n_audio_frames - max_len + 1, 0)
    return (audio_ids,)


get_audio_indexes(27, 2, 28)

In [ ]:
from safetensors import safe_open

path = "/fsx/rs2517/data/HDTF/audio/WRA_ToddYoung_000_wav2vec2_emb.safetensors"
tensors = {}
with safe_open(path, framework="pt") as f:
    tensor_slice = f.get_slice("audio")
    print(tensor_slice.get_shape())
tensor_slice[[0, 0, 1]]

In [ ]:
import decord

decord.bridge.set_bridge("torch")

vr = decord.VideoReader("/fsx/rs2517/data/HDTF/cropped_videos_original/WRA_ToddYoung_000.mp4")
print(len(vr))

In [ ]:
import glob

valid_files = 0
files = glob.glob("/fsx/rs2517/data/HDTF/cropped_videos_original/*.safetensors")
for file in files:
    with safe_open(file, framework="pt") as f:
        tensor_slice = f.get_slice("latents")
        latent_shape = tensor_slice.get_shape()[0]

    file = file.replace("cropped_videos_original", "audio").replace("video_512_latent", "wav2vec2_emb")
    with safe_open(file, framework="pt") as f:
        tensor_slice = f.get_slice("audio")
        audio_shape = tensor_slice.get_shape()[0]

    if latent_shape != audio_shape:
        print(
            f"latent_shape: {latent_shape}, audio_shape: {audio_shape}. Difference: {abs(latent_shape - audio_shape)}"
        )
    else:
        valid_files += 1
print(valid_files)

In [ ]:
import torch

scale = torch.linspace(5, 2, 14).unsqueeze(0)
scale

In [ ]:
audio_image_preds = [i for i in range(0, 15 * 25, 13)]
print(audio_image_preds)
len_audio_image_preds = len(audio_image_preds)
print(len_audio_image_preds)
for i in range(14, len_audio_image_preds + len_audio_image_preds % 14, 14):
    print(i)
    audio_image_preds.insert(i, 0)
print(audio_image_preds)

In [ ]:
import numpy as np

np.load("/fsx/behavioural_computing_data/face_generation_data/1000actors_nsv/audio_emb/G023_C009_10258J_001.npy").shape

import torch

torch.load(
    "/fsx/behavioural_computing_data/face_generation_data/1000actors_nsv/audio_emb/G023_C009_10258J_001_beats_emb.pt"
).shape

In [ ]:
[sample if ((i) % (14) != 0) or (i == 0) else "bite" for i, sample in enumerate(audio_image_preds)]

In [ ]:
from memory_profiler import profile

# Assuming these functions are defined to work with your specific file format
from safetensors import safe_open
from safetensors.torch import load_file

path = "/fsx/rs2517/data/HDTF/cropped_videos_original/WRA_VickyHartzler_000_video_512_latent.safetensors"


@profile
def method1():
    tensors = {}
    with safe_open(path, framework="pt") as f:
        tensor_slice = f.get_slice("latents")
    print("Method 1: Tensor slice loaded.")


@profile
def method2():
    tensor = load_file(path)["latents"]
    print("Method 2: Entire tensor loaded.")


method1()
method2()

In [ ]:
import torch


def logit_normal_sampler(m, s=1, beta_m=15, sample_num=1000000):
    y_samples = torch.randn_like(m) * s + m
    print(sample_num, s, (m + torch.randn(sample_num)).shape, torch.randn(sample_num).shape)
    x_samples = beta_m * (torch.exp(y_samples) / (1 + torch.exp(y_samples)))
    return x_samples


def mu_t(t, a=5, mu_max=1):
    t = t.to("cpu")
    return 2 * mu_max * t**a - mu_max


def get_sigma_s(t, a, beta_m):
    mu = mu_t(t, a=a)
    sigma_s = logit_normal_sampler(m=mu, sample_num=t.shape[0], beta_m=beta_m)
    return sigma_s


sigma = torch.randn(2, 1, 1, 1)

get_sigma_s(sigma, 5, 15).shape

In [2]:
import random

# Path to the file containing the video paths
file_path = "/data/home/antoni/datasets/filelist_videos_celebv_text.txt"


# Function to read the file and extract video paths
def read_video_paths(file_path):
    video_paths = []
    with open(file_path, "r") as file:
        for line in file:
            if "Duration:" in line and line.split(".")[0].isdigit():
                # Extract the video path from the line
                path = line.split("Path: ")[-1].strip()
                video_paths.append(path)
            else:
                video_paths.append(line.strip())
    return video_paths


# Read all video paths
all_video_paths = read_video_paths(file_path)

# Randomly select 100 video paths
selected_paths = random.sample(all_video_paths, min(100, len(all_video_paths)))

# Save the selected paths to a new file
output_file = "/data/home/antoni/code/generative-models/tmp.txt"
with open(output_file, "w") as file:
    for path in selected_paths:
        file.write(f"{path}\n")

print(f"Randomly selected 100 video paths have been saved to {output_file}")

Randomly selected 100 video paths have been saved to /data/home/antoni/code/generative-models/tmp.txt


In [6]:
import random

# Path to the file containing the video paths
file_path = "/data/home/antoni/datasets/filelist_video_mead.txt"

import os
import random


# Function to read the file and extract video paths
def read_video_paths(file_path):
    with open(file_path, "r") as file:
        return [line.strip() for line in file]


# Read all video paths
all_video_paths = read_video_paths(file_path)

# Shuffle the paths
random.shuffle(all_video_paths)

# Select and verify 500 existing paths
selected_paths = []
for path in all_video_paths:
    if os.path.exists(path.replace("video_crop", "video_crop_emb").replace(".mp4", "_video_512_latent.safetensors")):
        selected_paths.append(path)
        if len(selected_paths) == 500:
            break

# Check if we found 500 valid paths
if len(selected_paths) < 500:
    print(f"Warning: Only found {len(selected_paths)} valid paths")

# Save the selected paths to a new file
output_file = "/data/home/antoni/datasets/selected_500_video_paths_mead.txt"
with open(output_file, "w") as file:
    for path in selected_paths:
        file.write(f"{path}\n")

print(f"Randomly selected {len(selected_paths)} video paths have been saved to {output_file}")

# Display the first few paths
print("First few selected paths:")
print(selected_paths[:5])

Randomly selected 500 video paths have been saved to /data/home/antoni/datasets/selected_500_video_paths_mead.txt
First few selected paths:
['/fsx/behavioural_computing_data/face_generation_data/MEAD/video_crop/W016/video/front/fear/level_3/018.mp4', '/fsx/behavioural_computing_data/face_generation_data/MEAD/video_crop/M022/video/front/fear/level_3/024.mp4', '/fsx/behavioural_computing_data/face_generation_data/MEAD/video_crop/M013/video/front/surprised/level_3/027.mp4', '/fsx/behavioural_computing_data/face_generation_data/MEAD/video_crop/M009/video/front/sad/level_1/005.mp4', '/fsx/behavioural_computing_data/face_generation_data/MEAD/video_crop/M005/video/front/fear/level_2/006.mp4']


In [7]:
import os
from collections import Counter


# Function to extract emotion and level from path
def extract_emotion_and_level(path):
    parts = path.split("/")
    emotion = parts[-3]
    level = parts[-2]
    return emotion, level


# Count emotions and levels
emotion_counter = Counter()
level_counter = Counter()

for path in selected_paths:
    emotion, level = extract_emotion_and_level(path)
    emotion_counter[emotion] += 1
    level_counter[level] += 1

# Print results
print("Number of different emotions:")
for emotion, count in emotion_counter.items():
    print(f"{emotion}: {count}")

print("\nNumber of different levels:")
for level, count in level_counter.items():
    print(f"{level}: {count}")

print(f"\nTotal number of emotions: {len(emotion_counter)}")
print(f"Total number of levels: {len(level_counter)}")

Number of different emotions:
fear: 73
surprised: 72
sad: 68
happy: 66
disgusted: 61
angry: 63
contempt: 57
neutral: 40

Number of different levels:
level_3: 151
level_1: 186
level_2: 163

Total number of emotions: 8
Total number of levels: 3


In [27]:
keyframes = [
    None,
    0,
    13,
    26,
    39,
    52,
    65,
    78,
    91,
    104,
    117,
    130,
    143,
    156,
    None,
    169,
    182,
    195,
    208,
    221,
    234,
    247,
    260,
    273,
    286,
    299,
    312,
    325,
    None,
    338,
    351,
    364,
    377,
    390,
    403,
    416,
    429,
    442,
    455,
    468,
    481,
    494,
    None,
    507,
    520,
    533,
    546,
    559,
    572,
    585,
    598,
    611,
    624,
    637,
    650,
    663,
    None,
    676,
    689,
    702,
    715,
    728,
    741,
    754,
    767,
    780,
    793,
    806,
    819,
    832,
    None,
    845,
    858,
    871,
    884,
    897,
    910,
    923,
    936,
    949,
    962,
    975,
    988,
    1001,
    None,
    1014,
    1027,
    1040,
    1053,
    1066,
    1079,
]

combinations = [
    [0, 13],
    [13, 26],
    [26, 39],
    [39, 52],
    [52, 65],
    [65, 78],
    [78, 91],
    [91, 104],
    [104, 117],
    [117, 130],
    [130, 143],
    [143, 156],
    [156, 169],
    [169, 182],
    [182, 195],
    [195, 208],
    [208, 221],
    [221, 234],
    [234, 247],
    [247, 260],
    [260, 273],
    [273, 286],
    [286, 299],
    [299, 312],
    [312, 325],
    [325, 338],
    [338, 351],
    [351, 364],
    [364, 377],
    [377, 390],
    [390, 403],
    [403, 416],
    [416, 429],
    [429, 442],
    [442, 455],
    [455, 468],
    [468, 481],
    [481, 494],
    [494, 507],
    [507, 520],
    [520, 533],
    [533, 546],
    [546, 559],
    [559, 572],
    [572, 585],
    [585, 598],
    [598, 611],
    [611, 624],
    [624, 637],
    [637, 650],
    [650, 663],
    [663, 676],
    [676, 689],
    [689, 702],
    [702, 715],
    [715, 728],
    [728, 741],
    [741, 754],
    [754, 767],
    [767, 780],
    [780, 793],
    [793, 806],
    [806, 819],
    [819, 832],
    [832, 845],
    [845, 858],
    [858, 871],
    [871, 884],
    [884, 897],
    [897, 910],
    [910, 923],
    [923, 936],
    [936, 949],
    [949, 962],
    [962, 975],
    [975, 988],
    [988, 1001],
    [1001, 1014],
    [1014, 1027],
    [1027, 1040],
    [1040, 1053],
    [1053, 1066],
    [1066, 1079],
]

# Define the chunk size as a multiple of 14
chunk_size = 14  # You can change this to any multiple of 14
num_frames = 14

# Ensure the chunk size is a multiple of 14
if chunk_size % 14 != 0:
    raise ValueError("Chunk size must be a multiple of 14")

keyframes_list = [keyframes[i : i + num_frames] for i in range(0, len(keyframes), num_frames)]
print(keyframes_list)
chunk_size = 3
last_keyframe_prev = None
for i in range(0, len(keyframes_list), chunk_size):
    start = i
    end = i + chunk_size

    unflattened_keyframes = [elem for sublist in keyframes_list[start:end] for elem in sublist]

    if last_keyframe_prev is not None:
        unflattened_keyframes = [last_keyframe_prev] + unflattened_keyframes

    # Find the first non-None keyframe in the chunk
    first_keyframe = next((kf for kf in unflattened_keyframes if kf is not None), None)

    # Find the last non-None keyframe in the chunk
    last_keyframe = next((kf for kf in reversed(unflattened_keyframes) if kf is not None), None)

    # Store the last keyframe for the next iteration
    last_keyframe_prev = last_keyframe

    # Find the corresponding combinations and their indices
    start_idx = next((idx for idx, comb in enumerate(combinations) if comb[0] == first_keyframe), None)
    end_idx = next((idx for idx, comb in enumerate(reversed(combinations)) if comb[1] == last_keyframe), None)

    if start_idx is not None and end_idx is not None:
        end_idx = len(combinations) - 1 - end_idx  # Adjust for reversed enumeration

    print("-" * 100)
    print(first_keyframe, last_keyframe)
    print(combinations[start_idx : end_idx + 1][0], combinations[start_idx : end_idx + 1][-1])

    # If we've reached the end of the keyframes list, break the loop
    # if end >= len(keyframes_list):
    #     break

[[None, 0, 13, 26, 39, 52, 65, 78, 91, 104, 117, 130, 143, 156], [None, 169, 182, 195, 208, 221, 234, 247, 260, 273, 286, 299, 312, 325], [None, 338, 351, 364, 377, 390, 403, 416, 429, 442, 455, 468, 481, 494], [None, 507, 520, 533, 546, 559, 572, 585, 598, 611, 624, 637, 650, 663], [None, 676, 689, 702, 715, 728, 741, 754, 767, 780, 793, 806, 819, 832], [None, 845, 858, 871, 884, 897, 910, 923, 936, 949, 962, 975, 988, 1001], [None, 1014, 1027, 1040, 1053, 1066, 1079]]
----------------------------------------------------------------------------------------------------
0 494
[0, 13] [481, 494]
----------------------------------------------------------------------------------------------------
494 1001
[494, 507] [988, 1001]
----------------------------------------------------------------------------------------------------
1001 1079
[1001, 1014] [1066, 1079]


In [12]:
keyframes_chunks = [
    [None, 0, 13, 26, 39, 52, 65, 78, 91, 104, 117, 130, 143, 156],
    [None, 169, 182, 195, 208, 221, 234, 247, 260, 273, 286, 299, 312, 325],
    [None, 338, 351, 364, 377, 390, 403, 416, 429, 442, 455, 468, 481, 494],
    [None, 507, 520, 533, 546, 559, 572, 585, 598, 611, 624, 637, 650, 663],
    [None, 676, 689, 702, 715, 728, 741, 754, 767, 780, 793, 806, 819, 832],
    [None, 845, 858, 871, 884, 897, 910, 923, 936, 949, 962, 975, 988, 1001],
    [None, 1014, 1027, 1040, 1053, 1066, 1079],
]

combinations = [
    [0, 13],
    [13, 26],
    [26, 39],
    [39, 52],
    [52, 65],
    [65, 78],
    [78, 91],
    [91, 104],
    [104, 117],
    [117, 130],
    [130, 143],
    [143, 156],
    [156, 169],
    [169, 182],
    [182, 195],
    [195, 208],
    [208, 221],
    [221, 234],
    [234, 247],
    [247, 260],
    [260, 273],
    [273, 286],
    [286, 299],
    [299, 312],
    [312, 325],
    [325, 338],
    [338, 351],
    [351, 364],
    [364, 377],
    [377, 390],
    [390, 403],
    [403, 416],
    [416, 429],
    [429, 442],
    [442, 455],
    [455, 468],
    [468, 481],
    [481, 494],
    [494, 507],
    [507, 520],
    [520, 533],
    [533, 546],
    [546, 559],
    [559, 572],
    [572, 585],
    [585, 598],
    [598, 611],
    [611, 624],
    [624, 637],
    [637, 650],
    [650, 663],
    [663, 676],
    [676, 689],
    [689, 702],
    [702, 715],
    [715, 728],
    [728, 741],
    [741, 754],
    [754, 767],
    [767, 780],
    [780, 793],
    [793, 806],
    [806, 819],
    [819, 832],
    [832, 845],
    [845, 858],
    [858, 871],
    [871, 884],
    [884, 897],
    [897, 910],
    [910, 923],
    [923, 936],
    [936, 949],
    [949, 962],
    [962, 975],
    [975, 988],
    [988, 1001],
    [1001, 1014],
    [1014, 1027],
    [1027, 1040],
    [1040, 1053],
    [1053, 1066],
    [1066, 1079],
]

# Define the chunk size you want to process at once (must be a multiple of 14)
chunk_size = 28  # This example processes two chunks at a time (2 * 14 = 28)

# Ensure chunk size is a multiple of 14
if chunk_size % 14 != 0:
    raise ValueError("Chunk size must be a multiple of 14")

# Flatten the list of keyframes (since chunks are in separate lists)
flat_keyframes = [kf for chunk in keyframes_chunks for kf in chunk]

# Iterate through the flattened keyframes in blocks of 'chunk_size'
for i in range(0, len(flat_keyframes), chunk_size):
    keyframe_chunk = flat_keyframes[i : i + chunk_size]  # Get a chunk of the flattened keyframes
    valid_keyframes = [kf for kf in keyframe_chunk if kf is not None]  # Skip 'None'

    # Form pairs and match them with combinations
    for j in range(len(valid_keyframes) - 1):
        start_idx = valid_keyframes[j]
        end_idx = valid_keyframes[j + 1]

        # Calculate the combination index and ensure it's within bounds
        combination_idx = i // 14 + j
        if combination_idx < len(combinations):
            combination = combinations[combination_idx]
            print(f"Combination from keyframes: {start_idx} -> {end_idx} corresponds to interval: {combination}")

Combination from keyframes: 0 -> 13 corresponds to interval: [0, 13]
Combination from keyframes: 13 -> 26 corresponds to interval: [13, 26]
Combination from keyframes: 26 -> 39 corresponds to interval: [26, 39]
Combination from keyframes: 39 -> 52 corresponds to interval: [39, 52]
Combination from keyframes: 52 -> 65 corresponds to interval: [52, 65]
Combination from keyframes: 65 -> 78 corresponds to interval: [65, 78]
Combination from keyframes: 78 -> 91 corresponds to interval: [78, 91]
Combination from keyframes: 91 -> 104 corresponds to interval: [91, 104]
Combination from keyframes: 104 -> 117 corresponds to interval: [104, 117]
Combination from keyframes: 117 -> 130 corresponds to interval: [117, 130]
Combination from keyframes: 130 -> 143 corresponds to interval: [130, 143]
Combination from keyframes: 143 -> 156 corresponds to interval: [143, 156]
Combination from keyframes: 156 -> 169 corresponds to interval: [156, 169]
Combination from keyframes: 169 -> 182 corresponds to int